In [ ]:
Project Deliverable #3: Data Collection and Data Cleaning

In [ ]:
This following code loads our dataset and shows the first 5 rows. 


    

In [32]:
import pandas as pd

# Load my dataset
df = pd.read_csv("Firefox_bugs.csv")

# See the first few rows
df.head()


,Summary,Issue id,Status,Priority,Resolution,Created,Resolved,Description
0,Address bar doesn't elide origins correctly,1606532,RESOLVED,--,DUPLICATE,2020-01-01 05:10:54+00:00,2023-06-06 00:44:25+00:00,User Agent: Mozilla/5.0 (Windows NT 10.0; Win6...
1,"""TypeError: info.PDFFormatVersion is undefined...",1606566,VERIFIED,--,FIXED,2020-01-01 18:26:12+00:00,2020-02-06 10:01:56+00:00,When the PDF version cannot be extracted from ...
2,no suggestions while typing in urlbar,1606572,RESOLVED,--,WORKSFORME,2020-01-01 22:21:53+00:00,2020-01-14 13:08:25+00:00,User Agent: Mozilla/5.0 (X11; Linux x86_64; rv...
3,File association Remote Code Execution via com...,1606596,RESOLVED,--,FIXED,2020-01-02 07:54:43+00:00,2024-05-30 17:06:02+00:00,Tested on Microsoft Windows 10 Enterprise vers...
4,login button on faq page is not working,1606602,RESOLVED,--,INCOMPLETE,2020-01-02 10:11:03+00:00,2023-07-30 20:18:55+00:00,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...


In [ ]:
The following code counts the number of duplicate rows. 



    

In [3]:
# Count total duplicate rows
num_dupes = df.duplicated().sum()
print(f"Number of duplicate rows: {num_dupes}")

Number of duplicate rows: 0


In [ ]:
Since it showed no duplicates, we specified that the check for duplicated be based on duplicated summary and descriptions. 62 Rows were found. 



    

In [4]:
df[df.duplicated(subset=["Summary", "Description"])]

,Summary,Issue id,Status,Priority,Resolution,Created,Resolved,Description
2894,Perma browser_oneOffs_contextMenu.js | Test ti...,1651979,RESOLVED,--,DUPLICATE,2020-07-10 12:34:25+00:00,2020-07-22 20:41:37+00:00,[Tracking Requested - why for this release]:\n...
3073,Interface elements do not working sometimes af...,1654772,RESOLVED,--,DUPLICATE,2020-07-23 11:26:05+00:00,2020-07-28 20:47:16+00:00,User Agent: Mozilla/5.0 (Macintosh; Intel Mac ...
3863,Firefox ESR auto-update 68.12 -> 78 fail,1666791,RESOLVED,--,INVALID,2020-09-23 14:10:48+00:00,2020-09-23 16:45:33+00:00,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...
5784,can not search in chrome://browser/content/bro...,1692713,RESOLVED,--,DUPLICATE,2021-02-13 21:41:59+00:00,2021-02-13 21:43:25+00:00,User Agent: Mozilla/5.0 (Windows NT 10.0; Win6...
6084,Intermittent browser/components/privatebrowsin...,1696274,RESOLVED,P5,DUPLICATE,2021-03-03 21:37:41+00:00,2021-03-03 21:38:17+00:00,**Filed by:** abutkovits [at] mozilla.com\n**P...
...,...,...,...,...,...,...,...,...
23751,Intermittent browser/components/asrouter/tests...,1936842,RESOLVED,--,DUPLICATE,2024-12-12 15:58:29+00:00,2024-12-12 15:59:00+00:00,**Filed by:** sstanca [at] mozilla.com\n**Pars...
23969,Arbitrary Javascript injection in PDF.js throu...,1939692,VERIFIED,--,DUPLICATE,2025-01-02 08:50:15+00:00,2025-01-03 02:18:26+00:00,(Tested with Firefox 124.0b9 on Ubuntu and wit...
24294,severe browser lags and high CPU and RAM usage,1945048,RESOLVED,--,INCOMPLETE,2025-01-31 08:41:40+00:00,2025-03-04 07:07:16+00:00,User Agent: Mozilla/5.0 (Windows NT 10.0; Win6...
24385,Intermittent browser/components/messagepreview...,1946380,NEW,P1,NaN,2025-02-06 11:04:04+00:00,2025-03-03 06:34:31+00:00,**Filed by:** agoloman [at] mozilla.com\n**Par...


In [ ]:
This code removes all the duplicates found. 


    

In [5]:
# Remove all duplicate rows based on summary and description
df = df.drop_duplicates(subset=["Summary", "Description"])

In [6]:
df[df.duplicated(subset=["Summary", "Description"])]

,Summary,Issue id,Status,Priority,Resolution,Created,Resolved,Description


In [ ]:
The next two code excertps installs packages we need to complete our data pre-processing tasks




In [8]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [nltk]4/5 [nltk]b]


In [10]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [12]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 13.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 18.3 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [ ]:
The code encodes the status and resolution fields and shows results in the first few rows. 
We use oneHotEncoder for the resolution and status field because order doesnt matter.






In [16]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

encodCol = ohe.fit_transform(df[["Status", "Resolution"]])

df_encoded = pd.concat([
    df.reset_index(drop=True),
    pd.DataFrame(encodCol.astype(int), columns=ohe.get_feature_names_out(["Status", "Resolution"]))
], axis=1)

df_encoded.head()

,Summary,Issue id,Status,Priority,Resolution,Created,Resolved,Description,Status_ASSIGNED,Status_NEW,...,Status_VERIFIED,Resolution_DUPLICATE,Resolution_FIXED,Resolution_INACTIVE,Resolution_INCOMPLETE,Resolution_INVALID,Resolution_MOVED,Resolution_WONTFIX,Resolution_WORKSFORME,Resolution_nan
0,Address bar doesn't elide origins correctly,1606532,RESOLVED,--,DUPLICATE,2020-01-01 05:10:54+00:00,2023-06-06 00:44:25+00:00,User Agent: Mozilla/5.0 (Windows NT 10.0; Win6...,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"""TypeError: info.PDFFormatVersion is undefined...",1606566,VERIFIED,--,FIXED,2020-01-01 18:26:12+00:00,2020-02-06 10:01:56+00:00,When the PDF version cannot be extracted from ...,0,0,...,1,0,1,0,0,0,0,0,0,0
2,no suggestions while typing in urlbar,1606572,RESOLVED,--,WORKSFORME,2020-01-01 22:21:53+00:00,2020-01-14 13:08:25+00:00,User Agent: Mozilla/5.0 (X11; Linux x86_64; rv...,0,0,...,0,0,0,0,0,0,0,0,1,0
3,File association Remote Code Execution via com...,1606596,RESOLVED,--,FIXED,2020-01-02 07:54:43+00:00,2024-05-30 17:06:02+00:00,Tested on Microsoft Windows 10 Enterprise vers...,0,0,...,0,0,1,0,0,0,0,0,0,0
4,login button on faq page is not working,1606602,RESOLVED,--,INCOMPLETE,2020-01-02 10:11:03+00:00,2023-07-30 20:18:55+00:00,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
This next couple excerpts of code sets the ordinal order for the priority field with P5 being 
the most critical, encodes it and show results in the first couple rows
We dont use a label encoder because it will order alphabetically and in this case P5 is highest priority 
so we need to map the order manually to make sure it is right. 
We use pd.Categorical which is an ordinal encoder that preserves both label and order. 



In [17]:
priority_order = ["P5","P4","P3","P2","P1"]  # lowest → highest
priority_map = {p:i for i,p in enumerate(priority_order)}

df["Priority_encoded"] = df["Priority"].map(priority_map)

unmapped = df.loc[df["Priority_encoded"].isna(), "Priority"].unique()
print("Unmapped values:", unmapped)

Unmapped values: ['--']


In [18]:
df["Priority_cat"] = pd.Categorical(df["Priority"], categories=priority_order, ordered=True)

In [19]:
df[["Priority","Priority_encoded"]].head(10)

# check to make sure it worked
print(df["Priority_cat"].value_counts().sort_index())

Priority_cat
P5    4642
P4      57
P3    3787
P2    1822
P1    2323
Name: count, dtype: int64


In [20]:
df[["Priority", "Priority_cat", "Priority_encoded"]].head(10)

,Priority,Priority_cat,Priority_encoded
0,--,NaN,NaN
1,--,NaN,NaN
2,--,NaN,NaN
3,--,NaN,NaN
4,--,NaN,NaN
5,--,NaN,NaN
6,--,NaN,NaN
7,P3,P3,2.0
8,P5,P5,0.0
9,P5,P5,0.0


In [22]:
df.head(10)

,Summary,Issue id,Status,Priority,Resolution,Created,Resolved,Description,Priority_encoded,Priority_cat
0,Address bar doesn't elide origins correctly,1606532,RESOLVED,--,DUPLICATE,2020-01-01 05:10:54+00:00,2023-06-06 00:44:25+00:00,User Agent: Mozilla/5.0 (Windows NT 10.0; Win6...,NaN,NaN
1,"""TypeError: info.PDFFormatVersion is undefined...",1606566,VERIFIED,--,FIXED,2020-01-01 18:26:12+00:00,2020-02-06 10:01:56+00:00,When the PDF version cannot be extracted from ...,NaN,NaN
2,no suggestions while typing in urlbar,1606572,RESOLVED,--,WORKSFORME,2020-01-01 22:21:53+00:00,2020-01-14 13:08:25+00:00,User Agent: Mozilla/5.0 (X11; Linux x86_64; rv...,NaN,NaN
3,File association Remote Code Execution via com...,1606596,RESOLVED,--,FIXED,2020-01-02 07:54:43+00:00,2024-05-30 17:06:02+00:00,Tested on Microsoft Windows 10 Enterprise vers...,NaN,NaN
4,login button on faq page is not working,1606602,RESOLVED,--,INCOMPLETE,2020-01-02 10:11:03+00:00,2023-07-30 20:18:55+00:00,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...,NaN,NaN
5,dropdown list in search menu does not appear,1606605,RESOLVED,--,INCOMPLETE,2020-01-02 10:45:21+00:00,2020-02-13 16:04:38+00:00,User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...,NaN,NaN
6,right click context menu displaying in first s...,1606620,NEW,--,NaN,2020-01-02 12:46:36+00:00,2023-01-23 19:19:13+00:00,User Agent: Mozilla/5.0 (Macintosh; Intel Mac ...,NaN,NaN
7,Inefficient z-ordering of elements in the brow...,1606631,RESOLVED,P3,INCOMPLETE,2020-01-02 13:46:45+00:00,2024-06-28 09:36:11+00:00,Steps to reproduce:\n 1. Be on macOS.\n 2. Ena...,2.0,P3
8,pages with unsaved data should not be closed b...,1606644,NEW,P5,NaN,2020-01-02 14:46:03+00:00,2023-03-13 14:15:22+00:00,User Agent: Mozilla/5.0 (X11; Ubuntu; Linux x8...,0.0,P5
9,Intermittent browser/components/enterprisepoli...,1606648,RESOLVED,P5,INCOMPLETE,2020-01-02 14:57:57+00:00,2020-08-25 17:01:57+00:00,**Filed by:** nbeleuzu [at] mozilla.com\n**Par...,0.0,P5


In [ ]:
This code creates a function to process raw text by removing special characters, transforming to lowercase, tokenizing, stemming and removing stop words





In [24]:

EN_STOP = set(stopwords.words('english'))
STEMMER = PorterStemmer()

def preprocess_text(text: str):
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)

    # (d) Remove special characters 
    text = re.sub(r'[^A-Za-z0-9\s]', ' ', text)

    # Collapse multiple spaces if exists
    text = re.sub(r'\s+', ' ', text).strip()

    # (e) Lowercase
    text = text.lower()

    # (f) Tokenize
    tokens = word_tokenize(text)

    # (g) Remove stop words
    tokens = [t for t in tokens if t not in EN_STOP]

    # (h) Stemming
    tokens = [STEMMER.stem(t) for t in tokens]

    return tokens

In [ ]:
This code concatentes the text and description field into one column and displays the new column





In [26]:
# 3) Create the combined Text column
df["Summary"] = df["Summary"].fillna("").astype(str)
df["Description"] = df["Description"].fillna("").astype(str)
df["Text"] = (df["Summary"] + " " + df["Description"]).str.replace(r"\s+", " ", regex=True).str.strip()

# 4) Check to see if it worked
print("New column created:", "Text" in df.columns)
print(df[["Summary", "Description", "Text"]].head(5))

New column created: True
                                             Summary  \
0        Address bar doesn't elide origins correctly   
1  "TypeError: info.PDFFormatVersion is undefined...   
2              no suggestions while typing in urlbar   
3  File association Remote Code Execution via com...   
4            login button on faq page is not working   

                                         Description  \
0  User Agent: Mozilla/5.0 (Windows NT 10.0; Win6...   
1  When the PDF version cannot be extracted from ...   
2  User Agent: Mozilla/5.0 (X11; Linux x86_64; rv...   
3  Tested on Microsoft Windows 10 Enterprise vers...   
4  User Agent: Mozilla/5.0 (Windows NT 6.1; Win64...   

                                                Text  
0  Address bar doesn't elide origins correctly Us...  
1  "TypeError: info.PDFFormatVersion is undefined...  
2  no suggestions while typing in urlbar User Age...  
3  File association Remote Code Execution via com...  
4  login button on faq pag

In [ ]:
This code applies our preprocessing function to the newly created column and returns a column with processed tokens.


    

In [29]:
#Apply tokenization column
df["processed_tokens"] = df["Text"].apply(preprocess_text)
df["processed_text"]  = df["processed_tokens"].apply(lambda toks: " ".join(toks))

#Make sure it worked
df[["Text", "processed_tokens"]].head(5)

,Text,processed_tokens
0,Address bar doesn't elide origins correctly Us...,"[address, bar, elid, origin, correctli, user, ..."
1,"""TypeError: info.PDFFormatVersion is undefined...","[typeerror, info, pdfformatvers, undefin, pdf,..."
2,no suggestions while typing in urlbar User Age...,"[suggest, type, urlbar, user, agent, mozilla, ..."
3,File association Remote Code Execution via com...,"[file, associ, remot, code, execut, via, comma..."
4,login button on faq page is not working User A...,"[login, button, faq, page, work, user, agent, ..."
